In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import math
# https://github.com/FutureSharks/financial-data
# pip3 install https://github.com/FutureSharks/financial-data/archive/master.zip
import pyfinancialdata
import trade_functions

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.layers import Dense, Activation

In [ ]:
train_data = pyfinancialdata.get_multi_year(
    provider='oanda',
    instrument='GBP_USD',
    years=[2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014],
    time_group='12h',
    price_calculation='close',
    drop_non_price_columns=True,
)

test_data = pyfinancialdata.get_multi_year(
    provider='oanda',
    instrument='GBP_USD',
    years=[2015, 2016, 2017],
    time_group='12h',
    price_calculation='close',
    drop_non_price_columns=True,
)

train_data.plot(title='train data')
test_data.plot(title='test data')

## Train model

In [ ]:
def create_training_dataset(dataset, look_back, time_ahead):
    '''
    Converts an array of values into a dataset matrix of X and Y for a sequential model
    '''
    dataX, dataY = [], []

    for i in range(len(dataset) - look_back - time_ahead + 1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back + time_ahead - 1, 0])

    dataX = np.array(dataX)
    dataX = np.reshape(dataX, (dataX.shape[0], 1, dataX.shape[1]))
    dataY = np.array(dataY)

    return dataX, dataY

In [ ]:
# Create dataset from the DataFrame and normalise
dataset = train_data['price'].values
dataset = dataset.astype('float32')
dataset = dataset.reshape(-1, 1)
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
# Create X, Y for training
trainX, trainY = create_training_dataset(dataset, look_back=look_back, time_ahead=time_ahead)

In [ ]:
model.fit(trainX, trainY, epochs=epoch, batch_size=1, verbose=0)

## Test model

In [ ]:
# Create a scaler and fit
test_scaler = MinMaxScaler(feature_range=(0, 1))
test_scaler.fit(test_data['price'].values.reshape(-1, 1))

# a function to predict
def predict(values):
    values = test_scaler.transform(values.reshape(-1, 1))
    values = np.reshape(values, (1, 1, look_back))
    prediction = model.predict(values)
    prediction = prediction.reshape(-1, 1)
    return test_scaler.inverse_transform(prediction)

## Resources

* https://datascience.stackexchange.com/questions/76826/why-are-predictions-from-my-lstm-neural-network-lagging-behind-true-values